# Generating a Draft Graph Data Model with LLM Assistance
This notebook demonstrates a workflow and LLM prompts to generate a **draft** graph data model from tabular data in a .csv file. The goal is to assist a developer or product owner quickly identify in their tabular data:
* Nodes and their properties
* Relationships and their properties
* How the nodes are connected by relationships

If an LLM can analyze your data and propose a conceptual "80% solution" this can greatly accelerate development and time-to-value. Given that Neo4j is schemaless, it is relatively easy to update the graph database schema as you continue building the solution.  

*As always, be sure to verify any output from an LLM especially where it can impact a production system. While this notebook focuses on genrating ideas, for any code or technical details related to Neo4j always reference official Neo4j documentation.*





Notebook created by [Daniel Bukowski, Sales Engineer, Neo4j](https://www.linkedin.com/in/danieljbukowski/)

Last Updated: January 11, 2024

# Import Libraries

## General Libraries

In [26]:
# core libraries
import os
import time
import logging
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

import json
import seaborn as sns
import matplotlib.pyplot as plt
import re
import io

# library to protect secrets
import getpass

In [3]:
# libraries to display GCP LLM output
from IPython.display import Image, Markdown, display

In [4]:
# limit warnings
import warnings
warnings.filterwarnings("ignore")

## GCP Libraries

In [5]:
# # the following may be necessary to update your runtime
# # be sure to re-start the runtime after installing the updates
# !pip install vertexai --upgrade
# !pip install google-cloud-aiplatform --upgrade

# !pip install google --upgrade
# !pip install google.cloud --upgrade
# !pip install --upgrade google-cloud-language

In [6]:
# GCP and Colab authentication
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [7]:
# gcp and vertex AI
from google.cloud import aiplatform

import vertexai
from vertexai.preview.language_models import TextGenerationModel, ChatModel
from vertexai.preview.generative_models import GenerativeModel

# Authenticate to GCP
This will enable us to use additional VertexAI features.

In [9]:
# project ID if using GCP
PROJECT_ID = getpass.getpass()

··········


In [10]:
# region if using GCP or another cloud provider
REGION = getpass.getpass()

··········


In [11]:
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
# run the below in the jupyter terminal to authenticate your google account
# ! gcloud auth login

In [12]:
! gcloud config set project {PROJECT_ID}

Updated property [core/project].


# Instantiate GCP Gemini LLM

In [13]:
# function to print markdown returned by GCP models
def print_markdown(string):
    display(Markdown(string))

## GCP Gemini
[Link to documentation](https://cloud.google.com/vertex-ai/docs/generative-ai/multimodal/overview#gemini-api)


In [14]:
gemini = GenerativeModel("gemini-pro")

In [15]:
gemini_parameters = {
    "temperature": 0.2,
    "max_output_tokens": 8192,
    "top_p": 0.8,
    "top_k": 40,
}

# Read in PPP Loan Data
The following [public](https://data.sba.gov/dataset/ppp-foia) data includes US Government Paycheck Protection Program (PPP) loans over USD 150,000. The data has 53 features, which we will reduce to 14 for this initial workflow. Most knowledge graphs, especially for a POC, do not require all the data in a source database. Rather, 'less but focused and better' is often an ideal approach.

## PPP Data Over 150k

In [17]:
ppp_df = pd.read_csv('https://data.sba.gov/dataset/8aa276e2-6cab-4f86-aca4-a7dde42adf24/resource/738e639c-1fbf-4e16-beb0-a223831011e8/download/public_150k_plus_230930.csv')
ppp_df.shape

(968525, 53)

In [30]:
# set LoanNumber as a string because it is an identifier
ppp_df['LoanNumber'] = ppp_df['LoanNumber'].astype(str)

In [31]:
ppp_df.head()

,BorrowerName,BusinessType,LoanNumber,CurrentApprovalAmount,JobsReported,ProjectState,OriginatingLender,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED
0,"SUMTER COATINGS, INC.",Corporation,9547507704,769358.78,62.0,SC,Synovus Bank,NaN,769358.78,NaN,NaN,NaN,NaN,NaN
1,"PLEASANT PLACES, INC.",Sole Proprietorship,9777677704,736927.79,73.0,SC,Synovus Bank,NaN,736927.79,NaN,NaN,NaN,NaN,NaN
2,BOYER CHILDREN'S CLINIC,Non-Profit Organization,5791407702,691355.00,75.0,WA,"Bank of America, National Association",NaN,691355.00,NaN,NaN,NaN,NaN,NaN
3,KIRTLEY CONSTRUCTION INC,Corporation,6223567700,499871.00,21.0,CA,"Bank of America, National Association",NaN,499871.00,NaN,NaN,NaN,NaN,NaN
4,AERO BOX LLC,NaN,9662437702,367437.00,25.0,NaN,The Huntington National Bank,NaN,367437.00,NaN,NaN,NaN,NaN,NaN


In [ ]:
# down-select to the most relevant categories we will work with for now
ppp_features = ['BorrowerName',
                'BusinessType',
                'LoanNumber',
                'CurrentApprovalAmount',
                'JobsReported',
                'ProjectState',
                'OriginatingLender',
                'UTILITIES_PROCEED',
                'PAYROLL_PROCEED',
                'MORTGAGE_INTEREST_PROCEED',
                'RENT_PROCEED',
                'REFINANCE_EIDL_PROCEED',
                'HEALTH_CARE_PROCEED',
                'DEBT_INTEREST_PROCEED'
]

In [20]:
# subselect specific features
ppp_df = ppp_df[ppp_features].copy()
ppp_df.shape

(968525, 14)

# LLM-Assisted Knowledge Graph Generation

# Step One: Structured Data Understanding with LLM Assistance

In [41]:
# start new multi-turn chat
gemini_chat = gemini.start_chat()

In [42]:
prompt = """
Let's go step-by-step through the process of designing a graph
data model for my Neo4j database.

First, how can you help me understand my data as it relates to creating a graph
data model? What information about the data should I provide to you
as the LLM assistant?
"""

In [43]:
gemini_chat_resp = gemini_chat.send_message(
    content = prompt,
    generation_config = gemini_parameters,
    stream = False
)

print_markdown(gemini_chat_resp.text)

To help you understand your data as it relates to creating a graph data model, you can provide me with the following information:

1. **Entities and Their Attributes:**
   - Identify the key entities (objects) in your data. These could be people, places, products, or concepts.
   - List the attributes or properties associated with each entity. For example, for the entity "Person," attributes might include name, age, and location.

2. **Relationships Between Entities:**
   - Determine the relationships between different entities. These relationships can be one-to-one, one-to-many, or many-to-many.
   - Describe the nature of each relationship. For instance, a "Person" can have a "friend" relationship with another "Person," or a "Product" can have a "purchased_by" relationship with a "Customer."

3. **Cardinality of Relationships:**
   - Specify the cardinality of each relationship. Cardinality defines the number of entities that can participate in a relationship. It can be one-to-one, one-to-many, or many-to-many.

4. **Data Volume and Growth:**
   - Estimate the volume of data you currently have and anticipate in the future. This information helps determine the scalability requirements of your graph database.

5. **Query Patterns and Use Cases:**
   - Identify the common queries or use cases you expect to perform on your data. Understanding how you plan to use the data helps optimize the graph data model for efficient querying.

6. **Data Consistency and Integrity:**
   - Define any data consistency or integrity constraints that need to be enforced in the graph database. These constraints ensure the accuracy and reliability of your data.

7. **Security and Access Control:**
   - Specify the security requirements for accessing and managing the data in the graph database. This includes user roles, permissions, and authentication mechanisms.

8. **Data Evolution and Future Requirements:**
   - Consider potential changes or additions to your data in the future. A flexible graph data model should accommodate evolving data requirements.

By providing this information, you can help me better understand your data and design a graph data model that meets your specific needs and requirements.

## Ask LLM to Analyze Data

### Define Variables to Pass to LLM

In [44]:
# save .info() as a string variable
buffer = io.StringIO()
ppp_df.info(buf=buffer)

ppp_df_info = buffer.getvalue()

In [45]:
# save numeric .describe() as a variable
ppp_df_desc = ppp_df.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99])
ppp_df_desc

,CurrentApprovalAmount,JobsReported,UTILITIES_PROCEED,PAYROLL_PROCEED,MORTGAGE_INTEREST_PROCEED,RENT_PROCEED,REFINANCE_EIDL_PROCEED,HEALTH_CARE_PROCEED,DEBT_INTEREST_PROCEED
count,9.685250e+05,968524.000000,3.393690e+05,9.666930e+05,4.617200e+04,9.953000e+04,2.284800e+04,5.744100e+04,3.170700e+04
mean,5.306195e+05,51.886593,1.493001e+04,5.148761e+05,4.816586e+04,5.619698e+04,6.899911e+03,4.686451e+04,1.436319e+04
std,7.378860e+05,67.552670,8.493718e+04,7.140555e+05,1.591102e+05,1.113768e+05,6.215009e+04,1.029147e+05,5.550163e+04
min,1.500000e+05,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
10%,1.669620e+05,12.000000,1.000000e+00,1.625000e+05,0.000000e+00,4.095800e+03,0.000000e+00,1.000000e+00,0.000000e+00
25%,2.004000e+05,18.000000,1.000000e+00,1.956000e+05,0.000000e+00,1.353293e+04,0.000000e+00,8.000000e+03,0.000000e+00
50%,2.950000e+05,30.000000,1.000000e+00,2.870770e+05,1.000000e+04,2.887750e+04,0.000000e+00,1.944400e+04,5.000000e+02
75%,5.393829e+05,54.000000,4.500000e+03,5.246020e+05,4.025978e+04,5.625000e+04,0.000000e+00,4.400000e+04,9.915105e+03
90%,1.084455e+06,110.000000,3.074608e+04,1.052715e+06,1.078968e+05,1.166324e+05,0.000000e+00,1.020180e+05,3.187840e+04
95%,1.702404e+06,175.000000,6.320000e+04,1.649693e+06,2.030549e+05,1.950000e+05,1.000000e+04,1.836000e+05,6.237096e+04


In [46]:
# save categorical .describe() as a variable
ppp_df_desc_cat = ppp_df.describe(include='object')
ppp_df_desc_cat

,BorrowerName,BusinessType,LoanNumber,ProjectState,OriginatingLender
count,968521,967806,968525,968516,968525
unique,857334,25,968525,56,4236
top,FIRST UNITED METHODIST CHURCH,Corporation,9547507704,CA,"JPMorgan Chase Bank, National Association"
freq,40,418313,1,130614,53539


### Ask LLM to Analyze Data Summaries

In [47]:
prompt = f"""
You are a data scientist with experience creating Neo4j
graph data models from tabular data. I am a developer who will be
creating a Neo4j graph data model from the data in a single .csv file.

I want you to perform a preliminary analysis on my data to help us understand
its characteristics before we brainstorm about the graph data model.

The data in my .csv file contains information about financial loans made
to businesses.

The following is summary of the data features, data types, and missing values:
{ppp_df_info}

The following is a description of each feature in the data:
1. BorrowerName contains the name of the Business that applied for the loan.
It has the following distribution: {ppp_df_desc_cat['BorrowerName']}

2. BusinessType contains the type of business (i.e., Corp, Partnership, LLC, etc.),
It has the following distribution: {ppp_df_desc_cat['BusinessType']}

3. LoanNumber contains the unique identifier for the loan.

4. CurrentApprovalAmount contains the financial amount of the loan.
It has the following distribution: {ppp_df_desc['CurrentApprovalAmount']}

5. JobsReported contains the number of jobs the loan supports.
It has the following distribution: {ppp_df_desc['JobsReported']}

6. ProjectState contains the state where the funds will be used.
It has the following distribution: {ppp_df_desc_cat['ProjectState']}

7. OriginatingLender contains the lender that originated the loan.
It has the following distribution: {ppp_df_desc_cat['OriginatingLender']}

8. UTILITIES_PROCEED contains the amount of the loan the borrower said they will use to pay utilities.
It has the following distribution: {ppp_df_desc['UTILITIES_PROCEED']}

9. PAYROLL_PROCEED contains the amount of the loan the borrower said they will use for payroll.
It has the following distribution: {ppp_df_desc['PAYROLL_PROCEED']}

10. MORTGAGE_INTEREST_PROCEED contains the amount of the loan the borrower said they will use to pay mortgage interest.
It has the following distribution: {ppp_df_desc['MORTGAGE_INTEREST_PROCEED']}

11. RENT_PROCEED contains the amount of the loan the borrower said they will use to pay rent.
It has the following distribution: {ppp_df_desc['RENT_PROCEED']}

12. REFINANCE_EIDL_PROCEED contains the amount of the loan the borrower said they will use to refinance an existing loan.
It has the following distribution: {ppp_df_desc['REFINANCE_EIDL_PROCEED']}

13. HEALTH_CARE_PROCEED contains the amount of the loan the borrower said they will use to pay employee health care.
It has the following distribution: {ppp_df_desc['HEALTH_CARE_PROCEED']}

14. DEBT_INTEREST_PROCEED contains the amount of the loan the borrower said they will use to pay debt interest.
It has the following distribution: {ppp_df_desc['DEBT_INTEREST_PROCEED']}

Provide me with your preliminary analysis of this data. What are important
overall details about the data? What are the most important features?

Do not return your suggestion for the Neo4j graph data model
yet. We will do that in the next step.
"""

In [48]:
gemini_chat_resp = gemini_chat.send_message(
    content = prompt,
    generation_config = gemini_parameters,
    stream = False
)

print_markdown(gemini_chat_resp.text)

**Preliminary Analysis of the Loan Data:**

**Overall:**

* The dataset contains information about 968,525 financial loans made to businesses.
* There are 14 features in the dataset, including the loan amount, number of jobs supported, loan purpose, and lender information.
* Some features have missing values, such as `MORTGAGE_INTEREST_PROCEED` (95% missing) and `REFINANCE_EIDL_PROCEED` (97% missing).

**Important Details:**

* The total amount of loans approved is over $50 billion, with an average loan amount of $530,619.
* The majority of loans are for amounts between $200,000 and $500,000.
* The most common loan purpose is payroll, followed by utilities and rent.
* The top lender is JPMorgan Chase Bank, National Association, which originated over 5% of the loans.

**Most Important Features:**

* **Loan Amount:** This is the amount of money that was approved for the loan. It is a key factor in determining the size and scope of the project that the loan will be used for.
* **Number of Jobs Supported:** This is the number of jobs that the loan is expected to create or retain. It is an important measure of the economic impact of the loan.
* **Loan Purpose:** This is the purpose for which the loan will be used. It can be used to understand the needs of the businesses that are receiving loans.
* **Lender:** This is the financial institution that originated the loan. It can be used to track the distribution of loans across different lenders.

**Conclusion:**

The data provides valuable insights into the financial loans made to businesses. The preliminary analysis highlights the key features of the data and their importance in understanding the lending landscape. This information will be useful in designing a Neo4j graph data model that can effectively capture the relationships and patterns within the data.

In [49]:
prompt = f"""
That is a very helpful. Based upon your of the data in my .csv and your
knowledge of high-quality Neo4j graph data models, I would like you to return your
suggestion for translating the data in my .csv into a Neo4j graph data model.

Once built, the Neo4j graph will be used to identify
potential fraud. We have not identified fraudulent loans yet
and so do not have that information in this data.

Please return the following:
Suggested Nodes and their properties, along with your reasoning for each
Relationships and their properties, along with your reasoning for each
Include only nodes, relationships, and properties derived from
features from my .csv file.

Do not return any code to create the data model. I only want to
focus on the proposed nodes, relationships, and properties with
your explanation for why you suggested each.
"""

In [50]:
gemini_chat_resp = gemini_chat.send_message(
    content = prompt,
    generation_config = gemini_parameters,
    stream = False
)

print_markdown(gemini_chat_resp.text)

**Suggested Nodes:**

* **Business:**
    * Properties:
        * Name
        * Type
        * State
* **Loan:**
    * Properties:
        * Loan Number
        * Amount
        * Purpose
        * Approval Date
* **Lender:**
    * Properties:
        * Name
        * Location

**Relationships:**

* **APPLIED_FOR:**
    * From: Business
    * To: Loan
    * Properties:
        * Date
* **ORIGINATED:**
    * From: Lender
    * To: Loan
    * Properties:
        * Date
* **USED_FOR:**
    * From: Loan
    * To: Purpose
    * Properties:
        * Amount

**Reasoning:**

* **Nodes:**
    * **Business:** This node represents the business that applied for the loan. Its properties include the business name, type, and state.
    * **Loan:** This node represents the loan itself. Its properties include the loan number, amount, purpose, and approval date.
    * **Lender:** This node represents the financial institution that originated the loan. Its properties include the lender name and location.

* **Relationships:**
    * **APPLIED_FOR:** This relationship represents the application process for the loan. It connects a business to the loan that they applied for.
    * **ORIGINATED:** This relationship represents the origination of the loan. It connects a lender to the loan that they originated.
    * **USED_FOR:** This relationship represents the purpose of the loan. It connects a loan to the purpose for which it was used.

This data model captures the key entities and relationships in the loan data. It can be used to identify potential fraud by analyzing the connections between businesses, loans, and lenders. For example, you could identify businesses that have applied for multiple loans from different lenders in a short period of time, or lenders that have originated a large number of loans with missing or incomplete information.

You can further enhance the data model by adding additional nodes and relationships to capture more information from the data. For example, you could add a `Person` node to represent the individuals involved in the loan process, or a `Document` node to store loan-related documents.

In [51]:
prompt = f"""
That is a good start and very helpful.

Based on your experience building high-quality graph data
models, are there any improvements you would suggest?

For example, are there any node properties that should
be converted to separate, additional nodes in the data model?

Please return an updated graph data model with your suggested improvements.
Reference only features available in the original .csv file.

Do not return any code to create the data model. I only want to
focus on the proposed nodes, relationships, and properties.
"""

In [52]:
gemini_chat_resp = gemini_chat.send_message(
    content = prompt,
    generation_config = gemini_parameters,
    stream = False
)

print_markdown(gemini_chat_resp.text)

**Improved Graph Data Model:**

**Nodes:**

* **Business:**
    * Properties:
        * Name
        * Type
        * State
* **Loan:**
    * Properties:
        * Loan Number
        * Amount
        * Purpose
        * Approval Date
* **Lender:**
    * Properties:
        * Name
        * Location
* **Purpose:**
    * Properties:
        * Description

**Relationships:**

* **APPLIED_FOR:**
    * From: Business
    * To: Loan
    * Properties:
        * Date
* **ORIGINATED:**
    * From: Lender
    * To: Loan
    * Properties:
        * Date
* **USED_FOR:**
    * From: Loan
    * To: Purpose
    * Properties:
        * Amount

**Improvements:**

* Extracted the loan purpose into a separate `Purpose` node. This allows for more flexibility in representing the purpose of the loan, as it can now have multiple purposes.
* Added a `Description` property to the `Purpose` node to provide more context about the purpose of the loan.

This improved data model provides a more granular representation of the loan data, making it easier to analyze and identify potential fraud. For example, you could now identify loans that have been used for multiple purposes, or loans that have been originated by lenders that are known to be involved in fraudulent activities.

You can further enhance the data model by adding additional nodes and relationships to capture more information from the data. For example, you could add a `Person` node to represent the individuals involved in the loan process, or a `Document` node to store loan-related documents.

# Next Steps
This is an initial, draft workflow that I am continuing to improve. There are several next steps I plan to add, including:
* Asking the LLM how to prepare and pre-process the source data
* Asking the LLM how to best ingest the data into Neo4j
* Asking the LLM (multi-modal) to produce a visual graph data model
* Refactor the code using LangChain or LlamaIndex
* And others...

*As always, be sure to verify any output from an LLM especially where it can impact a production system. While this notebook focuses on genrating ideas, for any code or technical details related to Neo4j always reference official Neo4j documentation.*